# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

/Users/areena.arora/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/96.0.4664.45/chromedriver_mac64.zip
Driver has been saved in cache [/Users/areena.arora/.wdm/drivers/chromedriver/mac64/96.0.4664.45]
/var/folders/81/2s24wkdd02j9528vnjsx7m080000gn/T/ipykernel_77639/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [3]:
driver.get("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

In [4]:
driver.find_element(By.ID, 'pht_status').click()

In [5]:
driver.find_element(By.XPATH, '//*[@id="pht_status"]/option[10]').click()

In [6]:
driver.find_element(By.ID, 'pht_lnm').send_keys('Nguyen')

In [7]:
driver.find_element(By.XPATH, '//*[@id="dat-menu"]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[18]/td/input[1]').click()

## Scraping

Once you are on the results page, do this. **I'll guide you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't actual tabular data!

> You can use either Selenium by itself or Selenium+BeautifulSoup to scrape the results page. The choice is up to you!

### Loop through each result and print the entire row

Okay wait, maybe not, it's a heck of a lot of rows. Use `[:10]` to only do the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

> *Tip: If you're using Selenium, `By.TAG_NAME` is used if you don't have a class or ID. If you're using BeautifulSoup, just do your normal thing.*

In [ ]:
# table = driver.find_element(By.TAG_NAME, 'tbody')
# print(table.text)

In [8]:
all_names = driver.find_elements(By.TAG_NAME, 'tr')

for each_person in all_names[:11]:
    print(each_person.text)
    print('\n-----\n')

Name and Location Order Basis for Order

-----

NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.

-----

NGUYEN, DAI T
City: HOUSTON
County: Harris
Zip Code: 77034


License #: 765339

Complaint # COS20210005027 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,500. Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondent failed to store eyelash extensions in a sealed bag or covered container and kept in a clean d

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – `driver.find_element` or `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [9]:
each_person_name = driver.find_elements(By.TAG_NAME, 'tr')

for names in each_person_name[:11]:
    try:
        print(names.find_elements(By.CLASS_NAME, 'results_text')[0].text)
        print('--')
    except:
        print("Not a name")

Not a name
NGUYEN, THANH
--
NGUYEN, DAI T
--
NGUYEN, LONG D
--
NGUYEN, LUCIE HUONG
--
NGUYEN, CHINH
--
NGUYEN, JIMMY
--
NGUYEN, NAM
--
NGUYEN, DUC
--
NGUYEN, THU THAO THI
--
NGUYEN, MINH NHU
--


## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: If you're using Selenium by itself, you can get the HTML of something by doing `.get_attribute('innerHTML')` – that way it'll look like BeautifulSoup when you print it. It might help you diagnose your issue!*
> - *Tip: Or I guess you could just skip the one with the problem...*

In [10]:
each_person_violation = driver.find_elements(By.TAG_NAME, 'tr')
for violation in each_person_violation[:11]:
    try:
        print(violation.find_elements(By.TAG_NAME, 'td')[2].text)
        print('\n--\n')
    except:
        print("\nViolation not found\n--\n")


Violation not found
--

Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.

--

Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondent failed to store eyelash extensions in a sealed bag or covered container and kept in a clean dry debris-free storage area.

--

Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.

--

Respondent failed t

## Loop through each result, printing the complaint number

Output should look like this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

In [11]:
each_person_violation = driver.find_elements(By.TAG_NAME, 'tr')
for violation in each_person_violation[:11]:
    try:
        print(violation.find_elements(By.CLASS_NAME, 'results_text')[-2].text)
        print('--')
    except:
        print("Violation not found\n --")

Violation not found
 --
COS20210004784
--
COS20210005027
--
COS20210009745
--
COS20210011484
--
COS20210011721
--
COS20200007069
--
COS20210010530
--
COS20200007141
--
COS20200000839
--
COS20210009714
--


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [12]:
violations_details = driver.find_elements(By.TAG_NAME, 'tr')
violations_list  = []

for violations in violations_details[1:11]:
    violations_dict = {}
    violations_dict['person_name'] = violations.find_elements(By.CLASS_NAME, 'results_text')[0].text
    violations_dict['violation_details'] = violation.find_elements(By.TAG_NAME, 'td')[2].text
    violations_dict['violation_number'] = violation.find_elements(By.CLASS_NAME, 'results_text')[-2].text
    violations_dict['license_numbers'] = violations.find_elements(By.CLASS_NAME, 'results_text')[4].text
    violations_dict['zip_code'] = violations.find_elements(By.CLASS_NAME, 'results_text')[3].text
    violations_dict['county'] = violations.find_elements(By.CLASS_NAME, 'results_text')[2].text
    violations_dict['city'] = violations.find_elements(By.CLASS_NAME, 'results_text')[1].text
    violations_list.append(violations_dict)

violations_list


# print(name_results)
# print(violation_number)
# print(violation_details)
# print('License number =', license_numbers)
# print('Zip code:' ,zip_code)
# print('County:',county)
# print('City:',city)
# print("\n--\n")

[{'person_name': 'NGUYEN, THANH',
  'violation_details': "Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondent failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondent failed to follow proper sequential cleaning and disinfecting procedures when using portable whirlpool jets; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to maintain powdered alum, styptic powder, or a cyanoacrylate.",
  'violation_number': 'COS20210009714',
  'license_numbers': '790672',
  'zip_code': '75034',
  'county': 'COLLIN',
  'city': 'FRISCO'},
 {'person_name': 'NGUYEN, DAI T',
  'violation_details': "Respondent failed to fol

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

In [13]:
df = pd.DataFrame(violations_list)
df.head(3)

,person_name,violation_details,violation_number,license_numbers,zip_code,county,city
0,"NGUYEN, THANH",Respondent failed to follow whirlpool foot spa...,COS20210009714,790672,75034,COLLIN,FRISCO
1,"NGUYEN, DAI T",Respondent failed to follow whirlpool foot spa...,COS20210009714,765339,77034,Harris,HOUSTON
2,"NGUYEN, LONG D",Respondent failed to follow whirlpool foot spa...,COS20210009714,"760420, 1620583",76877,SAN SABA,SAN SABA


In [14]:
df.to_csv("texas_output.csv", index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [15]:
pd.read_csv('texas_output.csv')

,person_name,violation_details,violation_number,license_numbers,zip_code,county,city
0,"NGUYEN, THANH",Respondent failed to follow whirlpool foot spa...,COS20210009714,790672,75034,COLLIN,FRISCO
1,"NGUYEN, DAI T",Respondent failed to follow whirlpool foot spa...,COS20210009714,765339,77034,Harris,HOUSTON
2,"NGUYEN, LONG D",Respondent failed to follow whirlpool foot spa...,COS20210009714,"760420, 1620583",76877,SAN SABA,SAN SABA
3,"NGUYEN, LUCIE HUONG",Respondent failed to follow whirlpool foot spa...,COS20210009714,"762626, 1811788",78801,UVALDE,UVALDE
4,"NGUYEN, CHINH",Respondent failed to follow whirlpool foot spa...,COS20210009714,777067,76502,BELL,TEMPLE
5,"NGUYEN, JIMMY",Respondent failed to follow whirlpool foot spa...,COS20210009714,796773,75088,DALLAS,ROWLETT
6,"NGUYEN, NAM",Respondent failed to follow whirlpool foot spa...,COS20210009714,688039,77025,HARRIS,HOUSTON
7,"NGUYEN, DUC",Respondent failed to follow whirlpool foot spa...,COS20210009714,758793,79605,TAYLOR,ABILENE
8,"NGUYEN, THU THAO THI",Respondent failed to follow whirlpool foot spa...,COS20210009714,"802892, 1286737",78244,BEXAR,SAN ANTONIO
9,"NGUYEN, MINH NHU",Respondent failed to follow whirlpool foot spa...,COS20210009714,"766162, 1502172",75243,NaN,DALLAS
